In [ ]:
import os
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
import torch
import timm
from PIL import Image

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
sns.set_style('whitegrid')

## What did fine-tuning do the embeddings?

In [ ]:
n_samples = 8

In [ ]:
data_root = '../data/AI4EO-MapYourCity/v1/building-age-dataset/train/data/'
dev_df = pd.read_csv('../data/AI4EO-MapYourCity/splits/dev-set.csv')

In [ ]:
dev_embeddings_path = '/home/k/k202141/rootgit/AI4EO-MapYourCity/logs/evaluations/runs/dump/2024-05-04_21-12-42/'
os.listdir(dev_embeddings_path)[:5]

In [ ]:
city_df = dev_df.query('country_id == "QCD" and city_id == "4VAXW"')
_, samples = train_test_split(city_df, test_size=n_samples, stratify=city_df['label'], random_state=1312)

In [ ]:
samples.info()

In [ ]:
model = timm.create_model(
    'swinv2_small_window16_256.ms_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

In [ ]:
# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [ ]:
def load_photo(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        img.load()
        return img.convert("RGB")

In [ ]:
batch = None
dev_embeddings = []

for label, pid in zip(samples['label'], samples['pid']):
    img = transforms(load_photo(os.path.join(data_root, pid, 'street.jpg'))).unsqueeze(0)
    dev_embeddings.append(np.load(os.path.join(dev_embeddings_path, f'{pid}_streetview.npy')))
    
    if batch is None:
        batch = img
    else:
        batch = torch.cat([batch, img], 0)
        
dev_embeddings = np.asarray(dev_embeddings).squeeze()

In [ ]:
with torch.no_grad():
    embeddings = model(batch)
    embeddings = embeddings.numpy()

In [ ]:
embeddings.shape, np.asarray(dev_embeddings).shape

### Before and after fine-tuning

In [ ]:
k = 4
sns.jointplot(data=pd.DataFrame(dict(before=embeddings[k], after=dev_embeddings[k])),
              x='before', y='after');

## TSNE

### Read dev embeddings

In [ ]:
dev_embeddings = {'topview':[], 'streetview':[], 'sentinel2':[]}

for i, pid in enumerate(dev_df.pid.values):
    print(i / len(dev_df) * 100)
    for key in dev_embeddings:
        dev_embeddings[key].append(np.load(os.path.join(dev_embeddings_path, f'{pid}_{key}.npy')))

for key in dev_embeddings:
    dev_embeddings[key] = np.asarray(dev_embeddings[key]).squeeze()

In [ ]:
(dev_embeddings['topview']).shape

In [ ]:
tsne_topview = TSNE(perplexity=30)
tsne_streetview = TSNE(perplexity=30)
tsne_sentinel2 = TSNE(perplexity=30)

In [ ]:
%%time
tsne_components_topview = tsne_topview.fit_transform(dev_embeddings['topview']).T
tsne_components_streetview = tsne_streetview.fit_transform(dev_embeddings['streetview']).T
tsne_components_sentinel2 = tsne_sentinel2.fit_transform(dev_embeddings['sentinel2']).T

In [ ]:
labels = dev_df.label.values
centers_streetview = np.zeros([7, 2])
centers_topview = np.zeros([7, 2])
centers_sentinel2 = np.zeros([7, 2])

for i in range(7):
    
    ix = np.where(labels==i)[0]
    for j in range(2):
        centers_streetview[i,j] = np.mean(tsne_components_streetview[j][ix])
        centers_topview[i,j] = np.mean(tsne_components_topview[j][ix])
        centers_sentinel2[i,j] = np.mean(tsne_components_sentinel2[j][ix])

In [ ]:
centers_streetview

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(15, 5))

ax[0].scatter(tsne_components_topview[0], tsne_components_topview[1], c=labels, cmap='rainbow')
ax[1].scatter(tsne_components_streetview[0], tsne_components_streetview[1], c=labels, cmap='rainbow')
ax[2].scatter(tsne_components_sentinel2[0], tsne_components_sentinel2[1], c=labels, cmap='rainbow')

ax[0].set_title('Topview')
ax[1].set_title('Streetview')
ax[2].set_title('Sentinel-2')

ax[0].plot(centers_topview[:,0], centers_topview[:,1], 'kx', markersize=50)
ax[1].plot(centers_streetview[:,0], centers_streetview[:,1], 'kx', markersize=50)
ax[2].plot(centers_sentinel2[:,0], centers_sentinel2[:,1], 'kx', markersize=50)

plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

In [ ]:
km_streetview = KMeans(7, init=centers_streetview)
km_topview = KMeans(7, init=centers_topview)
km_sentinel2 = KMeans(7, init=centers_sentinel2)

In [ ]:
pred_streetview = km_streetview.fit_predict(tsne_components_streetview.T)
pred_topview = km_topview.fit_predict(tsne_components_topview.T)
pred_sentinel2 = km_sentinel2.fit_predict(tsne_components_sentinel2.T)

In [ ]:
print('streetview', confusion_matrix(pred_streetview, dev_df.label.values, normalize='true').diagonal().mean())
print('topview   ', confusion_matrix(pred_topview, dev_df.label.values, normalize='true').diagonal().mean())
print('sentinel2 ', confusion_matrix(pred_sentinel2, dev_df.label.values, normalize='true').diagonal().mean())

In [ ]:
fig, ax = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(10, 15))

# topview
ax[0,0].scatter(tsne_components_topview[0], tsne_components_topview[1], c=labels, cmap='rainbow')
ax[0,1].scatter(tsne_components_topview[0], tsne_components_topview[1], c=pred_topview, cmap='rainbow')

ax[1,0].scatter(tsne_components_streetview[0], tsne_components_streetview[1], c=labels, cmap='rainbow')
ax[1,1].scatter(tsne_components_streetview[0], tsne_components_streetview[1], c=pred_streetview, cmap='rainbow')

ax[2,0].scatter(tsne_components_sentinel2[0], tsne_components_sentinel2[1], c=labels, cmap='rainbow')
ax[2,1].scatter(tsne_components_sentinel2[0], tsne_components_sentinel2[1], c=pred_sentinel2, cmap='rainbow')

ax[0,0].set_title('True labels')
ax[0,1].set_title('Predicted clusters');

Idea is to use these clusters to judge test samples better where the models are unsure.
- Model predicts different label for each fold --> use this to decide